In [39]:
#Arxiv Researcg
# Import Tools Library
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [40]:
#usinh inbuilt tool of wiki
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
wiki.name

'wikipedia'

In [41]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=500)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
arxiv.name

'arxiv'

In [42]:
#Import RAG tools
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [43]:
loader = WebBaseLoader("https://www.ibm.com/think/topics/agentic-ai-vs-generative-ai")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)
vectorDb = FAISS.from_documents(documents=documents, embedding=OpenAIEmbeddings())
retriever = vectorDb.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023D28DB2CF0>, search_kwargs={})

In [44]:
#Convert the retriever into a tool
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "ibm_genai_search", "Search any information about GenAi or Agentic Ai")
retriever_tool.name

'ibm_genai_search'

In [45]:
tools = [wiki, arxiv, retriever_tool]
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\lalra\\anaconda3\\envs\\huggingface\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=500)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=500)),
 Tool(name='ibm_genai_search', description='Search any information about GenAi or Agentic Ai', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000023CDAC16F20>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x

In [46]:
# Run all the tools using Agents and LLM models
#Combine the tools with LLM-> AgentExecutor
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")

llm=ChatGroq(groq_api_key=groq_api_key, model="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000023D28AA08F0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000023D28F51400>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [47]:
# Now create out Prompt Template
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [48]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [49]:
#Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [50]:
response = agent_executor.invoke({"input": "Tell me about Agentic AI use cases in Healthcare"})
print(response) 



> Entering new AgentExecutor chain...

Invoking: `ibm_genai_search` with `{'query': 'Agentic AI use cases in Healthcare'}`


Agentic AI use cases


Customer service


The traditional models for customer chatbots had limitations due to the pre-programmed nature of the technology and would require human intervention at times. Whereas with autonomous agents, the model can quickly understand what a customer's intent and emotion is and take steps to resolve the issue.

These autonomous systems can therefore predictively assess a situation and help ensure a smoother customer interaction with a business. The customer experience is exceedingly important in the world today as businesses seek out higher retention and enhanced loyalty. Specifically, agentic AI can automate tedious tasks by gathering, cleaning, and formatting an organization’s data. These systems can take the weight off of human employees and free them up to do more high-impact projects and tasks.


Healthcare

Healthcare


AI t

In [51]:
print(response['output']) 

Agentic AI is poised to revolutionize healthcare by automating tasks, improving patient care, and streamlining workflows. 

One example mentioned is Propeller Health's smart inhaler, which uses agentic AI to collect real-time data from patients and alert healthcare providers when necessary.  This can help ensure patients are taking their medication correctly and identify potential issues early on.


Agentic AI can also be used to automate administrative tasks, freeing up healthcare professionals to focus on patient care.  For example, agentic AI could be used to schedule appointments, manage patient records, and process insurance claims.

However, the text also highlights the importance of cybersecurity in healthcare AI, as patient data is highly sensitive.  



In [52]:
response = agent_executor.invoke({"input": "What is attention is all you need"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'attention is all you need'}`


Published: 2024-07-22
Title: Attention Is All You Need But You Don't Need All Of It For Inference of Large Language Models
Authors: Georgy Tyukin, Gbetondji J-S Dovonon, Jean Kaddour, Pasquale Minervini
Summary: The inference demand for LLMs has skyrocketed in recent months, and serving
models with low latencies remains challenging due to the quadratic input length
complexity of the attention layers. In this work, we investigate the effect of
dropping MLP and attention layers at inference time o"Attention Is All You Need" is a groundbreaking paper in the field of natural language processing (NLP).  

It introduced the Transformer architecture, a novel neural network design that relies entirely on attention mechanisms to process sequential data like text. This architecture significantly outperformed previous models in tasks like machine translation, demonstrating the power of atten

In [53]:
print(response['output'])

"Attention Is All You Need" is a groundbreaking paper in the field of natural language processing (NLP).  

It introduced the Transformer architecture, a novel neural network design that relies entirely on attention mechanisms to process sequential data like text. This architecture significantly outperformed previous models in tasks like machine translation, demonstrating the power of attention in capturing long-range dependencies and relationships within text.  

The paper's core idea is that attention allows a model to focus on the most relevant parts of an input sequence when generating an output. This eliminates the need for recurrent networks, which process information sequentially and struggle with long sequences, and enables parallel processing, leading to faster training and inference. 


Let me know if you want to know more about Transformers or the "Attention Is All You Need" paper!



In [54]:
response = agent_executor.invoke({"input": "Who is lord Hanuman"})
print(response)



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Hanuman'}`


Page: Hanuman
Summary: Hanuman (; Sanskrit: हनुमान्, IAST: Hanumān), also known as Maruti, Bajrangabali, and Anjaneya, is a deity in Hinduism, revered as a divine vanara, and a devoted companion of the deity Rama. Central to the Ramayana, Hanuman is celebrated for his unwavering devotion to Rama and is considered a chiranjivi. He is traditionally believed to be the spiritual offspring of the wind deity Vayu, who is said to have played a significant role in his birth. In Shaiva tradition, he is rHanuman (; Sanskrit: हनुमान्, IAST: Hanumān), also known as Maruti, Bajrangabali, and Anjaneya, is a deity in Hinduism, revered as a divine vanara, and a devoted companion of the deity Rama. 


> Finished chain.
{'input': 'Who is lord Hanuman', 'output': 'Hanuman (; Sanskrit: हनुमान्, IAST: Hanumān), also known as Maruti, Bajrangabali, and Anjaneya, is a deity in Hinduism, revered as a divine vanara, and

In [55]:
print(response['output'])

Hanuman (; Sanskrit: हनुमान्, IAST: Hanumān), also known as Maruti, Bajrangabali, and Anjaneya, is a deity in Hinduism, revered as a divine vanara, and a devoted companion of the deity Rama. 

